In [1]:
#Installer des modules qu'on aurait besoin
!pip install pycountry
!pip install pays

     |████████████████████████████████| 10.0MB 2.8MB/s 
  Created wheel for pycountry: filename=pycountry-19.8.18-py2.py3-none-any.whl size=10627361 sha256=f3178d2a5f7d4de1c410e0b73a2fb2e80f91302a1cddc52da69afa5adaf36613
  Stored in directory: /root/.cache/pip/wheels/a2/98/bf/f0fa1c6bf8cf2cbdb750d583f84be51c2cd8272460b8b36bd3
Successfully built pycountry
     |████████████████████████████████| 112kB 2.8MB/s 
  Created wheel for pays: filename=pays-0.2.3-cp36-none-any.whl size=116955 sha256=a0a16ceb8dbb72fbdf67a629affd2c206a1fe931216f69a91baf9866f6377f3e
  Stored in directory: /root/.cache/pip/wheels/00/6b/52/3741cf41083453c269d6948d6716d84363e464fb5698ee5c6f
Successfully built pays


In [2]:
import pycountry as py
import pandas as pd
import re
import numpy as np
import nltk 
from nltk import PorterStemmer, download
from nltk.corpus import stopwords
from string import punctuation
download('stopwords')
import collections as c

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df_collection=pd.read_csv('/content/drive/My Drive/Colab Notebooks/collection_vv.csv')
df_article=pd.read_csv('/content/drive/My Drive/Colab Notebooks/documents_vv.csv')
df_auteur=pd.read_csv('/content/drive/My Drive/Colab Notebooks/personne_vv.csv')
df_redige=pd.read_csv('/content/drive/My Drive/Colab Notebooks/redige_vv.csv')

# Collection

In [5]:
df_collection.head()

,issn,nom_clt,nb_numeros,nb_document,pre_année_pub,der_année_pub,discpline,editeur
0,2262-3353,Hommes & Migrations,180.0,4523.0,1987.0,2008.0,"démographie, science politique",Musée de l'Histoire de l'Immigration
1,1950-6694,Revue économique,320.0,6776.0,1950.0,2002.0,économie,SciencesPo
2,1783-1768,Revue Philosophique de Louvain,441.0,12292.0,1894.0,2015.0,philosophie,Peeters
3,1957-7966,Population,320.0,12794.0,1946.0,2004.0,démographie,INED (Institut national d’études démographiques)
4,1777-5418,Revue Européenne des Migrations Internationales,49.0,814.0,1985.0,2001.0,démographie,MSH Poitiers


In [6]:
df_collection[df_collection['issn'].isnull()]

,issn,nom_clt,nb_numeros,nb_document,pre_année_pub,der_année_pub,discpline,editeur
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#Supprimer des collections non referencées
df_collection.drop([26,207,243], inplace=True)

In [0]:
df_collection=df_collection.fillna('')

In [10]:
df_collection['nb_numeros']=df_collection['nb_numeros'].astype(int)
df_collection['nb_document']=df_collection['nb_document'].astype(int)
df_collection['pre_année_pub']=df_collection['pre_année_pub'].astype(int)
df_collection['der_année_pub']=df_collection['der_année_pub'].astype(int)
df_collection.head()

,issn,nom_clt,nb_numeros,nb_document,pre_année_pub,der_année_pub,discpline,editeur
0,2262-3353,Hommes & Migrations,180,4523,1987,2008,"démographie, science politique",Musée de l'Histoire de l'Immigration
1,1950-6694,Revue économique,320,6776,1950,2002,économie,SciencesPo
2,1783-1768,Revue Philosophique de Louvain,441,12292,1894,2015,philosophie,Peeters
3,1957-7966,Population,320,12794,1946,2004,démographie,INED (Institut national d’études démographiques)
4,1777-5418,Revue Européenne des Migrations Internationales,49,814,1985,2001,démographie,MSH Poitiers


In [0]:
def verifie_doublons(df):
  list_=df.values.tolist()
  c_=c.Counter()
  for m in list_:
    c_[m]+=1
  m_=c_.most_common()
  lm=[]
  for m in m_:
    if m[1]<=1:
      break
    else:
      lm.append(m[0])
  return lm

In [86]:
verifie_doublons(df_collection['issn'])

[]

In [0]:
df_collection.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_collection.csv',index=False)

# Article


In [12]:
df_article[['num_doc','annee_publication']]=df_article[['num_doc','annee_publication']].astype(int)
df_article.head()

,num_doc,title,type,annee_publication,num_publication,abstract,issn
0,1,La fuite des cerveaux : le cas de l'ex-URSS,article,1992,1155,La fuite des cerveaux : le cas de l'ex-URSS......,2262-3353
1,2,"Croissance, redistribution et inégalités dans ...",article,1999,50-3,"Croissance, redistribution et inégalités dans ...",1950-6694
2,3,Que nous devons-nous les uns aux autres ? Le c...,article,2008,106-4,La fuite des cerveaux est-elle un phénomène in...,1783-1768
3,4,Chorafas Dimitri N. — La fuite des cerveaux. L...,compte-rendu,1970,25-5,Chorafas Dimitri N. — La fuite des cerveaux. L...,1957-7966
4,5,Au-delà de la « fuite des cerveaux » : la mobi...,article,1997,13-1,Au-delà de la « fuite des cerveaux » : la mobi...,1777-5418


In [18]:
df_article[df_article['num_doc'].isnull()]

,num_doc,title,type,annee_publication,num_publication,abstract,issn


In [0]:
df_article=df_article.fillna('')

In [0]:
t_article=df_article[['num_doc','title','annee_publication','issn']]
abstract=df_article[['num_doc','abstract']]

In [87]:
verifie_doublons(t_article['num_doc'])

[]

In [0]:
t_article.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_article.csv',index=False)

# Mot article

In [23]:
abstract.head()

,num_doc,abstract
0,1,La fuite des cerveaux : le cas de l'ex-URSS......
1,2,"Croissance, redistribution et inégalités dans ..."
2,3,La fuite des cerveaux est-elle un phénomène in...
3,4,Chorafas Dimitri N. — La fuite des cerveaux. L...
4,5,Au-delà de la « fuite des cerveaux » : la mobi...


In [0]:
def nettoyer_abstract(df):
       
    #Recuperer les stopswords 
    sr = nltk.corpus.stopwords.words('french')
    sc = ['cet','cette','celui','celle','celles','leurs','quoi','quel','lequel','laquelle','lesquels','lesquelles']
    stopwordss=list(set().union(sr,sc))
  
    
    #Recuperer les ponctuations
    poncts=punctuation
    poncts+='’'
    
    #lemmatisation
    stemmer=nltk.stem.snowball.FrenchStemmer()
    
    for i in df.index:
        doc=df.loc[i,['abstract']][0]
        clean_tokens=[]
        doc=str(doc)
        
        #Enlever les ponctuations
        for p in poncts :
            doc=doc.replace(p," ")
            
        doc=doc.lower()
        tokens=doc.split()

        #Enlever les stopwords et raciniser
        clean_tokens=[stemmer.stem(token) for token in tokens if (token not in stopwordss and len(token)>3)]

        #Enlever les chiffres
        clean_tokenss=[token for token in clean_tokens if token.isnumeric()==False]
        
        df.loc[i,['abstract']]=' '.join(clean_tokenss)
    
    return df

In [25]:
abstract=nettoyer_abstract(abstract)
abstract.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,num_doc,abstract
0,1,fuit cerveau urss exod inte¬ rieur cess accéle...
1,2,croissanc redistribu inégal model fuit cerveau...
2,3,fuit cerveau phénomen individuel légitim socia...
3,4,choraf dimitr fuit cerveau conquêt sav choraf ...
4,5,delà fuit cerveau mobil person haut qualifi ou...


In [26]:
vocabulaire=(' '.join(abstract['abstract'].values.tolist())).split()
vocabulaire[:5]

['fuit', 'cerveau', 'urss', 'exod', 'inte¬']

In [27]:
count_mot=c.Counter()
for mot in vocabulaire:
  count_mot[mot]+=1
len(count_mot)

39633

In [28]:
#Je prends les 10 mots les plus frequents comme mots clé
mots_cles=count_mot.most_common(12)
mots_cles

[('europ', 26814),
 ('fuit', 4275),
 ('des', 3823),
 ('plus', 3661),
 ('pay', 3105),
 ('cerveau', 2753),
 ('polit', 2382),
 ('franc', 2161),
 ('européen', 2101),
 ('tout', 2066),
 ('comm', 2058),
 ('entre', 1999)]

In [29]:
mots_cles=[mot[0] for mot in mots_cles]
mots_cles.remove('cerveau')
mots_cles.remove('fuit')
mots_cles

['europ',
 'des',
 'plus',
 'pay',
 'polit',
 'franc',
 'européen',
 'tout',
 'comm',
 'entre']

In [30]:
#Creer le dataframe mot_article
t_mot_article=pd.DataFrame(columns=['mot','num_article','frequence'])
j=0
for i in abstract.index:
  for mot in mots_cles:
    text=abstract.loc[i,['abstract']][0].split()
    if mot in text:
      freq=text.count(mot)
      t_mot_article.loc[j,:]=[mot,i+1,freq]
      j+=1
t_mot_article.head()

,mot,num_article,frequence
0,plus,2,1
1,polit,3,1
2,europ,4,1
3,plus,4,1
4,pay,4,1


In [0]:
t_mot_article.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_mot_article.csv',index=False)

#Pays article

In [0]:
#Une fonction qui renvoi la liste des pays du monde
def f_pays():
    list_pays=[]
    for pays in list(py.countries):
        a= re.search("name='"+'.+'+", numeric=",str(pays))
        if a:
            a=a.group(0)[6:-11]
            list_pays.append(a.lower())
    return list_pays

In [33]:
t_pays_article=pd.DataFrame(columns=['nom_pays','num_article'])
list_pays=f_pays()
j=0
for i in abstract.index:
    text=abstract.loc[i,['abstract']][0]
    for pays in list_pays:
      p=re.search(pays,text)
      if p:
        t_pays_article.loc[j,:]=[p.group(0),i+1]
        j+=1
t_pays_article.head()

,nom_pays,num_article
0,réunion,15
1,poland,36
2,oman,43
3,pakistan,62
4,egypt,66


In [0]:
t_pays_article.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_pays_article.csv',index=False)

# Auteur

In [42]:
df_auteur.head()

,num,nom,annee_naissance,annee_decès,bio
0,414356,Kortunov Andrei,NaN,NaN,NaN
1,140232,Bouyssou Rachel,NaN,NaN,NaN
2,24789,Docquier Frédéric,NaN,NaN,Docteur en sciences économiques. - Macroéconom...
3,61222,Rapoport Hillel,1963.0,NaN,NaN
4,731465,Chauvier Stéphane,1959.0,NaN,Maître de conférences de philosophie à l'Unive...


In [44]:
df_auteur[df_auteur['nom'].isnull()]

,num,nom,annee_naissance,annee_decès,bio


In [45]:
#Remplacer les Nan par vide
df_auteur=df_auteur.fillna('')
df_auteur.head()

,num,nom,annee_naissance,annee_decès,bio
0,414356,Kortunov Andrei,,,
1,140232,Bouyssou Rachel,,,
2,24789,Docquier Frédéric,,,Docteur en sciences économiques. - Macroéconom...
3,61222,Rapoport Hillel,1963,,
4,731465,Chauvier Stéphane,1959,,Maître de conférences de philosophie à l'Unive...


In [0]:
bios=df_auteur[['num','bio']]

In [47]:
#Ajouter les professions des auteurs
df_auteur['profession']=''
for i in df_auteur.index:
  bio=df_auteur.loc[i,['bio']][0]
  if bio!='':
    b=bio[:3]
    prof=re.search(r""+b+"\w*(['\(\) ]\w*)*[,\.-]",bio)
    if prof:
      df_auteur.loc[i,'profession']=prof.group(0)[:-1]
df_auteur.head()

,num,nom,annee_naissance,annee_decès,bio,profession
0,414356,Kortunov Andrei,,,,
1,140232,Bouyssou Rachel,,,,
2,24789,Docquier Frédéric,,,Docteur en sciences économiques. - Macroéconom...,Docteur en sciences économiques
3,61222,Rapoport Hillel,1963,,,
4,731465,Chauvier Stéphane,1959,,Maître de conférences de philosophie à l'Unive...,Maître de conférences de philosophie à l'Unive...


In [49]:
t_auteur=df_auteur[['num','nom','annee_naissance','annee_decès','profession']]
t_auteur.head()

,num,nom,annee_naissance,annee_decès,profession
0,414356,Kortunov Andrei,,,
1,140232,Bouyssou Rachel,,,
2,24789,Docquier Frédéric,,,Docteur en sciences économiques
3,61222,Rapoport Hillel,1963,,
4,731465,Chauvier Stéphane,1959,,Maître de conférences de philosophie à l'Unive...


In [88]:
verifie_doublons(t_auteur['num'])

[]

In [0]:
t_auteur.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_auteur.csv',index=False)

# Pays auteur


In [51]:
list_pays=f_pays()
bios['nom_pays']=''
for i in bios.index:
  bio=bios.loc[i,['bio']][0]
  if bio!='':
    bio=bio.lower()
    for pays in list_pays:
        pp=re.search(pays,bio)
        if pp:
          bios.loc[i,['nom_pays']]+=' '+pp.group(0)
bios.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remo

,num,bio,nom_pays
0,414356,,
1,140232,,
2,24789,Docteur en sciences économiques. - Macroéconom...,
3,61222,,
4,731465,Maître de conférences de philosophie à l'Unive...,


In [0]:
nationalite=bios[['num','nom_pays']]

In [55]:
t_nationalite=nationalite[nationalite['nom_pays']!='']
t_nationalite.head()

,num,nom_pays
18,33007,france
45,259994,france
50,249315,france
74,215901,france
103,234171,france


In [0]:
t_nationalite.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_nationalite.csv',index=False)

# Redige

In [57]:
df_redige.head()

,nom,num_doc,role
0,414356,1.0,NaN
1,140232,1.0,Traducteur
2,24789,2.0,NaN
3,61222,2.0,NaN
4,731465,3.0,NaN


In [0]:
df_redige['num_doc']=df_redige['num_doc'].astype(int)

In [60]:
t_redige=df_redige[['nom','num_doc']]
t_redige.head()

,nom,num_doc
0,414356,1
1,140232,1
2,24789,2
3,61222,2
4,731465,3


In [63]:
t_redige.rename(columns={'nom':'num'}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [0]:
t_redige.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_redige.csv',index=False)